# FalconX Assignment

## Assignment I - Basic Python - 3 points

In [21]:
from datetime import datetime, timedelta
import pandas as pd
import backtrader


## Variable declaration

In [46]:
today, numberOfDays = '2022-03-01', 3 
dataSet_url = 'https://data.binance.vision/data/spot/daily/trades/BTCUSDT/BTCUSDT-trades-'
csv_files = list()

## User Defined Functions

In [47]:
# Function to find out url address link from the given github link

def generateFileLinks(dataSet_url, csv_files,today, numberOfDays):
    ''' This function generate URL links for first 3 days of March month
        Paremeters: 
            URL - Intial url link
            csv_files(list) - empty list
            today - strat date
            numberOfDays - number of days to be read
        Returns:
            csv_files - return url address of all zip files   
    '''
    for i in range(3):
        current_date = str(datetime.strptime(today,'%Y-%m-%d') + timedelta(i)).replace(' 00:00:00','')
        csv_files.append(dataSet_url + str(current_date) + '.zip')
    return csv_files

In [53]:
def readcsv(file_path):
    return pd.read_csv(file_path, header=None)

def readZip(csv_files):
    ''' 
        This function read all url link into pandas dataframe
        Parameters:
            csv_files - list object contains url address of zip file for first 3 days of march month
            readcsv (user defined function) - Function filepath and read dataframe object
        Returns:
            data_df (dataframe) - Return combined data frame for all three dates
    '''
    data_df = pd.concat(map(readcsv, csv_files))
    return data_df


In [3]:
csv_files = generateFileLinks(dataSet_url, csv_files,today, numberOfDays)
data_df = readZip(csv_files)

In [55]:
data_df.head()

,0,1,2,3,4,5,6
0,1274557543,43160.00,0.00056,24.169600,1646092800000,True,True
1,1274557544,43160.01,0.00052,22.443205,1646092800003,False,True
2,1274557545,43160.01,0.00182,78.551218,1646092800004,False,True
3,1274557546,43160.01,0.00631,272.339663,1646092800005,False,True
4,1274557547,43160.00,0.00229,98.836400,1646092800006,True,True


## Assignment II - Simple Forward Looking Market Making Strategy - 5 points 

In [116]:
df =  data_df.loc[:,[0,4,1]]
df.rename(columns={0 : 'ID', 4 : 'time', 1 :'close'}, inplace=True)
df.time = pd.to_datetime(df.time,unit='ms')
df.close = df.close.astype(float)

In [ ]:
class TestStrategy(backtrader.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.order = None
     
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('Buy Executed ', order.executed.price)
            elif order.issell():
                self.log('Sell Executed ', order.executed.price)
            
            self.bar_executed = len(self)
        self.order = None
        
        
    def next(self):
        self.log('Close, %.2f' % self.dataclose[0]) 
        print(len(self))
        print(self.order)
        print(self.position)
        
        if self.dataclose[0]<self.dataclose[-1]:
            if self.dataclose[-1]<self.dataclose[-2]:
                self.log('Buy Create, %2f', self.dataclose[0])
                self.order = self.buy()
            else:
                if len(self)>=5000:
                    self.log('Sell Create, %2f',self.dataclose[0])
                    self.order = self.sell()
        

In [124]:
cerebro = backtrader.Cerebro()
cerebro.broker.set_cash(1000)
data = backtrader.feeds.PandasData(dataname=df)
cerebro.adddata(data)
cerebro.addsizer(backtrader.sizers.FixedSize, stake=1000)
cerebro.addstrategy(TestStrategy)
cerebro.run()
cerebro.broker.getvalue()

AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'

In [ ]:
# class MyStrategy():
#     quantity = 0
#     def __init__(self,lookback):
#         self.lookback = lookback
    
#     def buy(self, ask):
#         self.quantity+= ask
        
#     def sell(self, bid):
#         self.quantity-= bid
        
#     def hedge(self):
#         if self.quantity>5000:
#             self.quantity-=bid